In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 25 14:31:01 2019
@author: minyoungpark

Moved to Jupyter notebook (for better or worse) on 12/03/2020 by Joseph
Updated code as well to streamline 3D reconstruction
"""
import sys

# add calibration_folder to path
calib_folder = r'D:\Lab\GIT\proc-joe\Limblab_DLC\cam_calib_20200508'
sys.path.append(calib_folder)


from utils.utils import load_config
from calibration.intrinsic import calibrate_intrinsic
from calibration.extrinsic import calibrate_extrinsic
from triangulation.triangulate import reconstruct_3d
from utils.vis_utils import generate_three_dim_video
from utils.vis_utils import generate_three_dim_pictures
from utils.triangulation_utils import add_static_points
    
import numpy as np
import toml

In [8]:
# setup config file

parsed_toml = toml.load(r'D:\Lab\GIT\proc-joe\Limblab_DLC\cam_calib_20200508\config_Crackle_20200922.toml')


In [10]:
parsed_toml['paths_to_2d_data']

['C:\\\\Users\\\\dongq\\\\DeepLabCut\\\\Han-Qiwei-2020-09-22-RT3D\\\\videos\\\\exp_han_00009DLC_resnet50_HanSep22shuffle1_1030000filtered.csv',
 'C:\\\\Users\\\\dongq\\\\DeepLabCut\\\\Han-Qiwei-2020-09-22-RT3D\\\\videos\\\\exp_han_00010DLC_resnet50_HanSep22shuffle1_1030000filtered.csv',
 'C:\\\\Users\\\\dongq\\\\DeepLabCut\\\\Han-Qiwei-2020-09-22-RT3D\\\\videos\\\\exp_han_00011DLC_resnet50_HanSep22shuffle1_1030000filtered.csv',
 'C:\\\\Users\\\\dongq\\\\DeepLabCut\\\\Han-Qiwei-2020-09-22-RT3D\\\\videos\\\\exp_han_00012DLC_resnet50_HanSep22shuffle1_1030000filtered.csv']

In [ ]:
config = load_config('config_Test_20201123_15in.toml' )
#%% If you already ran calibration you don't need to run these.

calibrate_intrinsic(config)

calibrate_extrinsic(config)

In [ ]:
labels = ['pointX', 'pointY', 'pointZ']

snapshots = ['Han_20201123_test_15in00001DLC_resnet50_TestNov23shuffle1_1030000filtered.csv',
             'Han_20201123_test_15in00001DLC_resnet50_TestNov23shuffle1_1030000filtered.csv']

static = {'pointX': [[497.1956159,477.1799594],[286.2408452,338.6374849]],
          'pointY': [[602.8473412,420.1615679],[606.4223067,301.0989688]],
          'pointZ': [[493.8415928,182.0259331],[274.6480682,95.18916682]]}

In [ ]:
add_static_points(config, labels, static, snapshots)
# You can set 3D reconstruction output path through argument,
# reconstruct_3d(config, output_path='/media/minyoungpark/Min/1101')
# or skip it and set it in config file (reconstruction_output_folder_path).
recovery = reconstruct_3d(config)

In [ ]:
# =============================================================================
# #%% Save 3d recovery json file
# import numpy as np
# from json import JSONEncoder
# import json
# class NumpyArrayEncoder(JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return JSONEncoder.default(self, obj)
# with open("pop_0610_anipose.json", "w") as write_file:
#     json.dump(recovery, write_file, cls=NumpyArrayEncoder)
#     
# #%% Load 3d recovery json file
# import numpy as np
# from json import JSONEncoder
# import json
# with open("pop_0317_3.json", "r") as read_file:
#     print("Converting JSON encoded data into Numpy array")
#     recovery = json.load(read_file)
# recovery['registration_mat'] = np.array(recovery['registration_mat'])
# recovery['center'] = np.array(recovery['center'])
# =============================================================================


#%% Save 3d recovery json file
import numpy as np
from json import JSONEncoder
import json
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
with open("pop_0610_anipose.json", "w") as write_file:
    json.dump(recovery, write_file, cls=NumpyArrayEncoder)
#%% Load 3d recovery json file
import numpy as np
from json import JSONEncoder
import json
with open("pop_0317_3.json", "r") as read_file:
    print("Converting JSON encoded data into Numpy array")
    recovery = json.load(read_file)
recovery['registration_mat'] = np.array(recovery['registration_mat'])
recovery['center'] = np.array(recovery['center'])

#%% generate 3D picture to find the optimal azimuth and elevation value
generate_three_dim_pictures(config)

#%% Try 3D stick figure video

generate_three_dim_video(config)

#%% Testing if the generated 3D results makes sense, by calculating the distance between wrist and hand
"""
import pandas as pd
import numpy as np
from numpy import array as arr

data_path = 'C:/Users/dongq/DeepLabCut/Han-Qiwei-2020-02-21/3D-data/output_3d_data_rotate4.csv'

df = pd.read_csv(data_path)
wrist = np.empty((len(df), 3))
hand = np.empty((len(df), 3))

wrist[:,0] = arr(df['wrist1_x'])
wrist[:,1] = arr(df['wrist1_y'])
wrist[:,2] = arr(df['wrist1_z'])


hand[:,0] = arr(df['hand2_x'])
hand[:,1] = arr(df['hand2_y'])
hand[:,2] = arr(df['hand2_z'])

dist = wrist - hand
dist_finite = dist[np.isfinite(dist[:,0]), :]
dist_3d = np.linalg.norm(dist_finite, axis=1)
print(np.median(dist_3d))
"""